In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [2]:
sc = SparkContext()
ssc = StreamingContext(sc, 5)

In [3]:
lines = ssc.socketTextStream("localhost", 9999)

In [4]:
type(lines)

pyspark.streaming.dstream.DStream

# Echo streaming

### Inizializzo lo stream da terminale con l'istruzione nc -lk 9999

In [5]:
# lines.pprint()

In [6]:
# ssc.start()
# ssc.awaitTermination()

# Words streaming

In [7]:
# words = lines.flatMap(lambda text: text.split())
# words.pprint()

In [8]:
#ssc.start()
#ssc.awaitTermination()

# Keywords monitoring

In [9]:
keywords = ["beer", "craft"]

lines.flatMap(lambda text: text.split()) \
        .filter(lambda word: (word.lower() in keywords)) \
        .map(lambda word: (word.lower(), 1)) \
        .reduceByKey(lambda a,b: a+b) \
        .pprint()

In [10]:
#ssc.start()
#ssc.awaitTermination()

# Registrare una tabella SQL temporanea

In [11]:
from pyspark.sql import SQLContext

# sqlContext = SQLContext(sc)

In [12]:
from pyspark.sql import Row

def get_sql_context(spark_context):
    if('sqlContext' not in globals()):
        globals()['sqlContext'] = SQLContext(spark_context)
    return globals()['sqlContext']

def process_rdd(time, rdd):
    if(rdd.count()>0):
        row_rdd = rdd.map(lambda w: Row(name=w[0], count=w[1]))
        sqlContext = get_sql_context(rdd.context)
        df = sqlContext.createDataFrame(row_rdd)
        df.createOrReplaceTempView("popularity")

In [13]:
def aggregate_count(new_value, total_sum):
    add_1 = sum(new_values)
    add_2 = (total_sum or 0)
    return add_1 + add_2

In [14]:
keywords = ["udemy", "coursera", "edx", "udacity"]

lines.flatMap(lambda text: text.split()) \
        .filter(lambda word: (word.lower() in keywords)) \
        .map(lambda word: (word.lower(), 1)) \
        .updateStateByKey(aggregate_count) \
        .foreachRDD(process_rdd)

In [15]:
ssc.checkpoint("/home/sparkmachine/Project")
ssc.start()

-------------------------------------------
Time: 2020-03-25 07:45:05
-------------------------------------------

-------------------------------------------
Time: 2020-03-25 07:45:10
-------------------------------------------

-------------------------------------------
Time: 2020-03-25 07:45:15
-------------------------------------------

-------------------------------------------
Time: 2020-03-25 07:45:20
-------------------------------------------

-------------------------------------------
Time: 2020-03-25 07:45:25
-------------------------------------------
('udemy', 1)

-------------------------------------------
Time: 2020-03-25 07:45:30
-------------------------------------------

-------------------------------------------
Time: 2020-03-25 07:45:35
-------------------------------------------

